# Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
import os 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import dask.dataframe as dd 

### Notes: 
    a) I worked on two approaches: first with sample dataset and second with entire dataset, both included in this notebook.
    b) I intentionally leave some bad code snippets on the notebook just to document failed attempts as well.
    c) Take this notebook also as the final report.

## First aproach: sample dataset

### Getting file

In [4]:
df = pd.read_csv('data/sample.csv', sep=',', decimal = '.',parse_dates=['fechaRegistro'])
df.head()

,producto,presentacion,marca,categoria,catalogo,precio,fechaRegistro,cadenaComercial,giro,nombreComercial,direccion,estado,municipio,latitud,longitud
0,CUADERNO FORMA ITALIANA,96 HOJAS PASTA DURA. CUADRICULA CHICA,ESTRELLA,MATERIAL ESCOLAR,UTILES ESCOLARES,25.9,2011-05-18,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
1,CRAYONES,CAJA 12 CERAS. JUMBO. C.B. 201423,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,27.5,2011-05-18,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
2,CRAYONES,CAJA 12 CERAS. TAMANO REGULAR C.B. 201034,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,13.9,2011-05-18,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
3,COLORES DE MADERA,CAJA 12 PIEZAS LARGO. TRIANGULAR. C.B. 640646,PINCELIN,MATERIAL ESCOLAR,UTILES ESCOLARES,46.9,2011-05-18,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
4,COLOR LARGO,CAJA 36 PIEZAS. CON SACAPUNTAS. 68-4036,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,115.0,2011-05-18,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417


### Verifying null values existence

In [5]:
df.isnull().sum()

producto             0
presentacion         0
marca                0
categoria            0
catalogo             0
precio               0
fechaRegistro        0
cadenaComercial      0
giro                 0
nombreComercial      0
direccion            0
estado               0
municipio            0
latitud            770
longitud           770
dtype: int64

#### There are some null values in latitud and longitud columns but it are not used to answer the question so we will ignore it for now.

### How many commercial chains are monitored, and therefore, included in this database?

In [5]:
df.groupby(['cadenaComercial'])['cadenaComercial'].count()

cadenaComercial
ABASTECEDORA LUMEN       9
COMERCIAL MEXICANA    1508
I.S.S.S.T.E.           195
WAL-MART               287
Name: cadenaComercial, dtype: int64

#### Answer to question: 4

### What are the top 10 monitored products by State?

In [58]:
df.groupby(['estado','producto']).agg(noItems=('producto', 'count')).reset_index().groupby(['estado']).apply(lambda x: x.sort_values(['noItems'],ascending = False)).reset_index(drop=True).groupby(['estado']).head(10).set_index(['estado','producto'])

noItems
estado           producto                        
DISTRITO FEDERAL DETERGENTE P/ROPA             25
                 LECHE ULTRAPASTEURIZADA       23
                 REFRESCO                      22
                 CHILES EN LATA                21
                 MAYONESA                      20
                 SHAMPOO                       18
                 TELEVISORES                   17
                 GALLETAS DULCES               16
                 JABON DE TOCADOR              16
                 PAN DE CAJA                   14
MÉXICO           REFRESCO                      26
                 DETERGENTE P/ROPA             24
                 SHAMPOO                       19
                 CHILES EN LATA                18
                 TOALLA FEMENINA               17
                 MAYONESA                      17
                 LECHE ULTRAPASTEURIZADA       17
                 YOGHURT                       15
                 DESODORANTE                   15
                 TELEVISORES                   15

#### Answer to question: Entire table

### Which is the commercial chain with the highest number of monitored products?

In [82]:
df.groupby(['cadenaComercial'])['producto'].count().sort_values().tail(1)



cadenaComercial
COMERCIAL MEXICANA    1508
Name: producto, dtype: int64

#### Answer to question: Comercial Mexicana

## Second Approach: entire dataset

### I made some attempts to load the dataset but my pc can't handle it, those attempts are 1) and 2), and finally third attempt was the final solution.

1) ( don't run it if you don't want to see your computer die ;D ) 

In [ ]:
dfFull = pd.read_csv('data/all_data.csv', sep=',', decimal = '.',parse_dates=['fechaRegistro'])
dfFull.head()

2) i tried using chunks but was not enough (don't execute, please continue to next solution :) )

In [ ]:
dfFull=[]
with pd.read_csv('data/all_data.csv', sep=',', decimal = '.',parse_dates=['fechaRegistro'], chunksize=1000000) as reader:
    for chunk in reader:
        dfFull.append(chunk)
df = pd.concat(dfFull,sort=False)
df.head(5)

Since the entire dataset is so big, maybe next tasks could help:
 - 1) Use parallelization.
 - 2) Reduce and transform dataset if it's possible.

The implementation of first task was realized using dask framework to parallelize the processing of entire dataset:

In the second task I took next decisions:

    a) keep the three indispensable columns to answer the questions: (cadenaComercial, estado, producto).
    b) very hard decision: discard column presentacion, In the dataset it is clearly seen that there are products that have several presentations, so probably question 2 should include the grouping by state, product and presentation, however, I discard it because the question only asks to group by product.
    c) discard columns direccion, latitud, longitud and municipio because it aren't relevant to answer the questions and we already have estado column to represent the location in a general way.
    d) discard column nombreComercial because represents almost same information that cadenaComercial.
    e) discard columns catalogo and giro because aren't relevant to answer the questions. 
    f) keep categoria and marca because i'd like to know which are the most monitorized products by categoria and marca.
    g) discard fechaRegistro column because is not relevant to answer the questions, this column could be very interesting if we treat this problem as a time series but it's out of scope for this exercise.

In [10]:
dfFull = dd.read_csv('data/all_data.csv', sep=',', decimal = '.',usecols=['categoria','cadenaComercial','estado','producto','marca'])
dfFull.head()

,producto,presentacion,marca,categoria,cadenaComercial,estado
0,CUADERNO FORMA ITALIANA,96 HOJAS PASTA DURA. CUADRICULA CHICA,ESTRELLA,MATERIAL ESCOLAR,ABASTECEDORA LUMEN,DISTRITO FEDERAL
1,CRAYONES,CAJA 12 CERAS. JUMBO. C.B. 201423,CRAYOLA,MATERIAL ESCOLAR,ABASTECEDORA LUMEN,DISTRITO FEDERAL
2,CRAYONES,CAJA 12 CERAS. TAMANO REGULAR C.B. 201034,CRAYOLA,MATERIAL ESCOLAR,ABASTECEDORA LUMEN,DISTRITO FEDERAL
3,COLORES DE MADERA,CAJA 12 PIEZAS LARGO. TRIANGULAR. C.B. 640646,PINCELIN,MATERIAL ESCOLAR,ABASTECEDORA LUMEN,DISTRITO FEDERAL
4,COLOR LARGO,CAJA 36 PIEZAS. CON SACAPUNTAS. 68-4036,CRAYOLA,MATERIAL ESCOLAR,ABASTECEDORA LUMEN,DISTRITO FEDERAL


### 1) How many commercial chains are monitored, and therefore, included in this database?

In [7]:
dfFull.groupby(['cadenaComercial'])['cadenaComercial'].count().compute()

cadenaComercial
7 ELEVEN                                    55946
ABARROTERA DE TLAXCALA                      10941
ABARROTERA GUADALUPANA (FRUTAS)             50422
ABARROTERA SANCHEZ                          29425
ABARROTES ARTES                              4655
                                            ...  
MERCADO  GENERAL EMILIANO ZAPATA SALAZAR      313
PESCADERIA EL MALECON                          17
BOTICA KENT TLAXCALA                           73
MUEBLERIA ALMACENES DEL NORTE                  21
MINISUPER MERCADO TOGUZ                        78
Name: cadenaComercial, Length: 705, dtype: int64

#### Answer to question: 705

### 2) What are the top 10 monitored products by State?

In [8]:
custom_count = dd.Aggregation(
    name='noItems',
    chunk=lambda s0: s0.count(),
    agg=lambda s0: s0.count()
)  

dfA = dfFull[['estado','producto']].set_index(['estado','producto'])
dfA.groupby(['estado','producto']).agg(custom_count).compute()


NotImplementedError: Dask dataframe does not yet support multi-indexes.
You tried to index with this index: ['estado', 'producto']
Indexes must be single columns only.

#### The error indicates that we can't use dask with multi-index so i'll try to solve this problem with pandas dataframes

In [3]:
dfA = pd.read_csv('data/all_data.csv', sep=',', decimal = '.',usecols=['estado','producto'])
dfA = dfA.groupby(['estado','producto']).agg(noItems=('producto', 'count')).reset_index().groupby(['estado']).apply(lambda x: x.sort_values(['noItems'],ascending = False)).reset_index(drop=True).groupby(['estado']).head(10).set_index(['estado','producto'])

In [4]:
dfA

noItems
estado         producto                        
AGUASCALIENTES FUD                        12005
               DETERGENTE P/ROPA          10188
               LECHE ULTRAPASTEURIZADA     9824
               SHAMPOO                     9654
               REFRESCO                    9481
...                                         ...
ZACATECAS      SHAMPOO                    15012
               CHILES EN LATA             14866
               COMPONENTES DE AUDIO       14799
               REFRESCO                   13925
estado         producto                      20

[321 rows x 1 columns]

#### Answer to question: Entire table above.
#### Note: We can see that there are trash rows with values 'estado' and 'producto' literally so we can consider to drop those values in a future iteration of data cleaning process.

### 3) Which is the commercial chain with the highest number of monitored products?

In [7]:
dfFull.groupby(['cadenaComercial'])['producto'].count().compute().sort_values().tail(1)



cadenaComercial
WAL-MART    8643133
Name: producto, dtype: int64

#### Answer to question: Walmart

### 4) Use data to find an interest fact

#### I'd like to know what are the three most reliable brands of each category to know what should i choose when i need to buy some product. I am going to assume that the most monitored products are the ones with the most complaints, so I will look for the brands that have the least monitored products to define them as the most reliable.

In [11]:
dfB = pd.read_csv('data/all_data.csv', sep=',', decimal = '.',usecols=['categoria','marca'])
dfB = dfB.groupby(['categoria','marca']).agg(noItems=('marca', 'count')).reset_index().groupby(['categoria']).apply(lambda x: x.sort_values(['noItems'],ascending = True)).reset_index(drop=True).groupby(['categoria']).head(3).set_index(['categoria','marca'])

In [13]:
dfB

noItems
categoria                         marca                       
ACCESORIOS DOMESTICOS             GOLDEN HILLS              71
                                  PHILIPS                   75
                                  MARCA INSTITUCIONAL      540
ACEITES Y GRASAS VEG. COMESTIBLES IBERIA                     4
                                  COCINERA                 199
...                                                        ...
UTENSILIOS DOMESTICOS             OSTER                    755
VINOS Y LICORES                   MONOPOLE                   2
                                  SANTA EMILIANA             7
                                  RISCAL S.L.               12
categoria                         marca                     20

[106 rows x 1 columns]

### 5. What are the lessons learned from this exercise?

####  I learned:
    a) when we have big datasets we need to use more sophisticated techniques to process it, after some failed attempts to process with common techniques (load all dataset in memory), i had to use dask to parallelize the processing and optimize memory.
    b) limitation of dask, it doesn't work with multi-index
    c) is very important to do many iterations of data cleaning in big datasets, it's very hard to find all the useless data in only one iteration, i found it near the end of the exercise. In this exercise I did many tests and I discovered that columns 'precio' and 'fechaRegistro' needed treatment because it has different incorrect formats.
    d) sometimes the decisions made can be difficult as discard the column presentacion, i'm not sure if i should have done it.

### 6. Can you identify other ways to approach this problem? Explain.


Use architectures where there is batch processing for this type of large datasets and streaming processing for real-time data processing